## pre

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install matplotlib==3.4.3 d2l

     |████████████████████████████████| 10.3 MB 6.7 MB/s 
     |████████████████████████████████| 82 kB 772 kB/s 
     |████████████████████████████████| 9.9 MB 61.3 MB/s 
     |████████████████████████████████| 20.1 MB 63.6 MB/s 
     |████████████████████████████████| 61 kB 9.4 MB/s 
     |████████████████████████████████| 83 kB 2.4 MB/s 
     |████████████████████████████████| 83 kB 2.0 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:
import collections
import math
import os
import shutil
import pandas as pd
import torch
import torchvision
from torch import nn
from d2l import torch as d2l

## 获取并组织数据集

下载数据集

为了便于入门，我们提供包含前1000个训练图像和5个随机测试图像的数据集的小规模样本。 要使用Kaggle竞赛的完整数据集，你需要将以下demo变量设置为False

In [2]:
# d2l.DATA_HUB['cifar10_tiny'] = (d2l.DATA_URL + 'kaggle_cifar10_tiny.zip',
#                                 '2068874e4b9a9f0fb07ebe0ad2b29754449ccacd')

# 如果你使用完整的Kaggle竞赛的数据集，设置demo为False
demo = False

if demo:
    data_dir = d2l.download_extract('cifar10_tiny')
else:
    data_dir = '/content/drive/MyDrive/Colab Notebooks/data/cifar-10'

整理数据集

In [3]:
%cd /content/drive/MyDrive/'Colab Notebooks'/computer_vision

/content/drive/MyDrive/Colab Notebooks/computer_vision


In [4]:
def read_csv_labels(fname):
    """读取fname来给标签字典返回一个文件名"""
    with open(fname, 'r') as f:
        # 跳过文件头行(列名)
        lines = f.readlines()[1:]
    tokens = [l.rstrip().split(',') for l in lines]
    return dict(((name, label) for name, label in tokens))

labels = read_csv_labels(os.path.join(data_dir, 'trainLabels.csv'))
print('# 训练样本 :', len(labels))
print('# 类别 :', len(set(labels.values())))

# 训练样本 : 50000
# 类别 : 10


定义reorg_train_valid函数来将验证集从原始的训练集中拆分出来

In [5]:
def copyfile(filename, target_dir):
    """将文件复制到目标目录"""
    os.makedirs(target_dir, exist_ok=True)
    shutil.copy(filename, target_dir)

def reorg_train_valid(data_dir, labels, valid_ratio):
    """将验证集从原始的训练集中拆分出来"""
    # 训练数据集中样本最少的类别中的样本数
    n = collections.Counter(labels.values()).most_common()[-1][1]
    # 验证集中每个类别的样本数
    n_valid_per_label = max(1, math.floor(n * valid_ratio))
    label_count = {}
    for train_file in os.listdir(os.path.join(data_dir, 'train')):
        label = labels[train_file.split('.')[0]]
        fname = os.path.join(data_dir, 'train', train_file)
        copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                     'train_valid', label))
        if label not in label_count or label_count[label] < n_valid_per_label:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                         'valid', label))
            label_count[label] = label_count.get(label, 0) + 1
        else:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                         'train', label))
    return n_valid_per_label

下面的reorg_test函数用来在预测期间整理测试集，以方便读取

In [6]:
def reorg_test(data_dir):
    """在预测期间整理测试集，以方便读取"""
    for test_file in os.listdir(os.path.join(data_dir, 'test')):
        copyfile(os.path.join(data_dir, 'test', test_file),
                 os.path.join(data_dir, 'train_valid_test', 'test',
                              'unknown'))

使用一个函数来调用前面定义的函数read_csv_labels、reorg_train_valid和reorg_test

In [16]:
def reorg_cifar10_data(data_dir, valid_ratio):
    labels = read_csv_labels(os.path.join(data_dir, 'trainLabels.csv'))
    reorg_train_valid(data_dir, labels, valid_ratio)
    reorg_test(data_dir)

In [17]:
batch_size = 32 if demo else 128
valid_ratio = 0.1
reorg_cifar10_data(data_dir, valid_ratio)

## 图像增广

In [7]:
transform_train = torchvision.transforms.Compose([
    # 在高度和宽度上将图像放大到40像素的正方形
    torchvision.transforms.Resize(40),
    # 随机裁剪出一个高度和宽度均为40像素的正方形图像，
    # 生成一个面积为原始图像面积0.64到1倍的小正方形，
    # 然后将其缩放为高度和宽度均为32像素的正方形
    torchvision.transforms.RandomResizedCrop(32, scale=(0.64, 1.0),
                                                   ratio=(1.0, 1.0)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    # 标准化图像的每个通道
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])])

In [8]:
transform_test = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])])

## 读取数据集

In [52]:
import py7zr
archive = py7zr.SevenZipFile(f'/content/drive/MyDrive/Colab Notebooks/data/cifar-10/test.7z', mode='r')
archive.extractall(path=f'/content/drive/MyDrive/Colab Notebooks/data/cifar-10/train_valid_test/test')
archive.close()

In [46]:
train_ds, train_valid_ds = [torchvision.datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', folder),
    transform=transform_train) for folder in ['train', 'train_valid']]

valid_ds = torchvision.datasets.ImageFolder(os.path.join(data_dir, 'train_valid_test', 'valid'),transform=transform_test)

In [53]:
test_ds = torchvision.datasets.ImageFolder(os.path.join(data_dir, 'train_valid_test', 'test'),transform=transform_test)

In [55]:
train_iter, train_valid_iter = [torch.utils.data.DataLoader(
    dataset, batch_size, shuffle=True, drop_last=True)
    for dataset in (train_ds, train_valid_ds)]

valid_iter = torch.utils.data.DataLoader(valid_ds, batch_size, shuffle=False,
                                         drop_last=True)

test_iter = torch.utils.data.DataLoader(test_ds, batch_size, shuffle=False,
                                        drop_last=False)

## 定义模型

In [56]:
def get_net():
    num_classes = 10
    net = d2l.resnet18(num_classes, 3)
    return net
loss = nn.CrossEntropyLoss(reduction="none")

## 定义训练函数

In [57]:
def train(net, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_period,
          lr_decay):
    trainer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9,
                              weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.StepLR(trainer, lr_period, lr_decay)
    num_batches, timer = len(train_iter), d2l.Timer()
    legend = ['train loss', 'train acc']
    if valid_iter is not None:
        legend.append('valid acc')
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
                            legend=legend)
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])
    for epoch in range(num_epochs):
        net.train()
        metric = d2l.Accumulator(3)
        for i, (features, labels) in enumerate(train_iter):
            timer.start()
            l, acc = d2l.train_batch_ch13(net, features, labels,
                                          loss, trainer, devices)
            metric.add(l, acc, labels.shape[0])
            timer.stop()
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (metric[0] / metric[2], metric[1] / metric[2],
                              None))
        if valid_iter is not None:
            valid_acc = d2l.evaluate_accuracy_gpu(net, valid_iter)
            animator.add(epoch + 1, (None, None, valid_acc))
        scheduler.step()
    measures = (f'train loss {metric[0] / metric[2]:.3f}, '
                f'train acc {metric[1] / metric[2]:.3f}')
    if valid_iter is not None:
        measures += f', valid acc {valid_acc:.3f}'
    print(measures + f'\n{metric[2] * num_epochs / timer.sum():.1f}'
          f' examples/sec on {str(devices)}')

## 训练和验证模型

In [58]:
devices, num_epochs, lr, wd = d2l.try_all_gpus(), 20, 2e-4, 5e-4
lr_period, lr_decay= 4, 0.9
net = get_net()
train(net, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_period, lr_decay)

## 在 Kaggle 上对测试集进行分类并提交结果

In [59]:
for i, (features, labels) in enumerate(train_valid_iter):
  print(labels)
  break

tensor([7, 9, 9, 7, 9, 7, 1, 5, 0, 6, 4, 6, 2, 5, 5, 8, 0, 2, 8, 7, 9, 0, 4, 6,
        6, 7, 5, 2, 8, 8, 8, 0, 9, 4, 6, 9, 3, 9, 7, 6, 2, 5, 1, 2, 4, 3, 4, 2,
        3, 8, 6, 6, 2, 9, 1, 9, 6, 8, 1, 8, 6, 3, 8, 0, 7, 5, 9, 8, 6, 1, 6, 6,
        7, 4, 8, 9, 2, 2, 4, 8, 2, 3, 2, 8, 0, 2, 4, 7, 7, 4, 9, 6, 8, 1, 8, 3,
        2, 1, 6, 2, 5, 7, 6, 4, 3, 7, 0, 0, 3, 7, 5, 3, 0, 9, 1, 9, 1, 2, 0, 7,
        7, 8, 4, 8, 1, 5, 9, 5])


In [60]:
net_1= get_net()
preds = []
train(net_1, train_valid_iter, None, num_epochs, lr, wd, devices, lr_period,
      lr_decay)

In [66]:
for X, _ in test_iter:
    y_hat = net_1(X.to(devices[0]))
    preds.extend(y_hat.argmax(dim=1).type(torch.int32).cpu().numpy())
sorted_ids = list(range(1, len(test_ds) + 1))
sorted_ids.sort(key=lambda x: str(x))
df = pd.DataFrame({'id': sorted_ids, 'label': preds})
df['label'] = df['label'].apply(lambda x: train_valid_ds.classes[x])
df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/cifar-10/result/submission.csv', index=False)

In [ ]:
torch.save(net,'/content/drive/MyDrive/Colab Notebooks/data/cifar-10/model/net.pkl')

In [ ]:
net = torch.load('/content/drive/MyDrive/Colab Notebooks/data/cifar-10/model/net.pkl')

In [ ]:
torch.save(net_1,'/content/drive/MyDrive/Colab Notebooks/data/cifar-10/model/net_1.pkl')

In [65]:
net_1 = torch.load('/content/drive/MyDrive/Colab Notebooks/data/cifar-10/model/net_1.pkl')